In [2]:
import pandas as pd
from os import path
import os
import copy

path_partA = "C:\\Users\\bm-\\Downloads\\Parte A\\Parte A"

**a)	Su primera tarea es construir un DataFrame que incluya todos aquellos stocks que se encuentren dentro del listado NASDAQ.txt y que no corresponden al listado excluded_symbols.txt. Dicho DataFrame debe contener una nueva columna llamada “symbol” que contenga el nombre del stock asociado a cada uno de los registros.**

In [3]:
#---a
path_full_history = path.join(path_partA, "full_history")

df_nasdaq = pd.read_csv(path.join(path_partA, "NASDAQ.txt"), sep="\t", header=0)
df_excluded_sym = pd.read_csv(path.join(path_partA, "excluded_symbols.txt"), header=None)

stock = []

files_stock = []
for fname in os.listdir(path_full_history):
    if os.path.isfile(os.path.join(path_full_history, fname)): 
        files_stock.append(os.path.splitext(fname)[0])

for i in range(len(df_nasdaq)):
    if df_nasdaq.loc[i, "Symbol"] in files_stock:
        stock.append(df_nasdaq.loc[i, "Symbol"])

for i in range(len(df_excluded_sym)):
    if df_excluded_sym.loc[i, 0] in copy.copy(stock):
        stock.remove(df_excluded_sym.loc[i,0])

df_a = pd.DataFrame()
count = 0
for i in range(len(stock)):
    df_tmp = pd.read_csv(path.join(path_full_history, stock[i] + ".csv"), sep=",", header=0)
    df_tmp["symbol"] = stock[i]
    df_a = pd.concat([df_a, df_tmp], ignore_index=True)
    count += 1

In [4]:
print(count)
print(len(stock))
print(len(df_a))

3410
3410
10182789


In [5]:
print(df_a.head(),"\n")
print(df_a.dtypes)

         date    volume       open      close       high        low  \
0  2019-04-18  12545500  76.129997  75.980003  76.129997  75.339996   
1  2019-04-17   2595100  75.699997  76.110001  76.199997  75.260002   
2  2019-04-16   4131200  74.879997      75.18  75.300003  74.470001   
3  2019-04-15   7181000  76.599998  74.370003  76.599998  74.260002   
4  2019-04-12   4768700  75.599998      76.68  77.029999  75.599998   

    adjclose symbol  
0  75.980003   AABA  
1  76.110001   AABA  
2      75.18   AABA  
3  74.370003   AABA  
4      76.68   AABA   

date        object
volume      object
open        object
close       object
high        object
low         object
adjclose    object
symbol      object
dtype: object


In [6]:
df_a1 = df_a.copy()    #########

**b)	Posterior a ello realice las transformaciones de datos para que se encuentren en el formato correcto.**

In [7]:
#---b
df_a1["date"] = pd.to_datetime(df_a1["date"], infer_datetime_format=True)
df_a1["volume"] = df_a1["volume"].astype(int)
df_a1["open"] = df_a1["open"].astype(float)
df_a1["close"] = df_a1["close"].astype(float)
df_a1["high"] = df_a1["high"].astype(float)
df_a1["low"] = df_a1["low"].astype(float)
df_a1["adjclose"] = df_a1["adjclose"].astype(float)
print(df_a1.dtypes)

date        datetime64[ns]
volume               int32
open               float64
close              float64
high               float64
low                float64
adjclose           float64
symbol              object
dtype: object


**c)	Mediante la API de Banxico lea el tipo de cambio (Tipo de cambio para solventar obligaciones denominadas en moneda extranjera Fecha de determinación (FIX)).**

In [8]:
#---c
from urllib.request import urlopen
import json

with urlopen("https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43718/datos?"+
"token=1728d77b21b641c1704df48ee020b179966b4100f187bc9c2bc83d0f671ad83b") as response:
    resp = json.loads(response.read().decode("utf8"))

data = resp["bmx"]["series"][0]["datos"]
df_tiposCambio = pd.DataFrame(data)
df_tiposCambio["fecha"] = pd.to_datetime(df_tiposCambio["fecha"], format="%d/%m/%Y")
df_tiposCambio["dato"] = pd.to_numeric(df_tiposCambio["dato"], downcast="float")

In [9]:
df_a2 = df_a1.copy()    #########

**d)	Transforme los valores monetarios de dólares a pesos (Ocupe el promedio mensual de los tipos de cambio para ello).**

In [10]:
#---d
means_month = df_tiposCambio.groupby(pd.PeriodIndex(df_tiposCambio["fecha"], freq="M"))["dato"].mean().reset_index()

for i, row in means_month.iterrows():
    df_a2.loc[(pd.DatetimeIndex(df_a2["date"]).month == means_month.loc[i, "fecha"].month) & (pd.DatetimeIndex(df_a2["date"]).year == means_month.loc[i, "fecha"].year), ["open","close","high","low","adjclose"]] = \
        df_a2[(pd.DatetimeIndex(df_a2["date"]).month == means_month.loc[i, "fecha"].month) & (pd.DatetimeIndex(df_a2["date"]).year == means_month.loc[i, "fecha"].year)][["open","close","high","low","adjclose"]]\
            .apply(lambda x: x*row["dato"])

In [12]:
print(means_month,"\n")
print(df_a2.dtypes)

       fecha       dato
0    1991-11   3.068377
1    1991-12   3.070921
2    1992-01   3.067723
3    1992-02   3.062905
4    1992-03   3.066768
..       ...        ...
363  2022-02  20.449520
364  2022-03  20.556210
365  2022-04  20.108805
366  2022-05  20.030527
367  2022-06  20.018509

[368 rows x 2 columns] 

date        datetime64[ns]
volume               int32
open               float64
close              float64
high               float64
low                float64
adjclose           float64
symbol              object
dtype: object


In [13]:
print(means_month[means_month["fecha"] == "2019-03"])
print("-"*7)
print(df_a[(pd.DatetimeIndex(df_a["date"]).month == 3) & (pd.DatetimeIndex(df_a["date"]).year == 2019) & (pd.DatetimeIndex(df_a["date"]).day == 29) & (df_a["symbol"]=="AABA")][["date","open","close","high","low","adjclose","symbol"]])
print("-"*7)
print(df_a2[(pd.DatetimeIndex(df_a2["date"]).month == 3) & (pd.DatetimeIndex(df_a2["date"]).year == 2019) & (pd.DatetimeIndex(df_a2["date"]).day == 29) & (df_a2["symbol"]=="AABA")][["date","open","close","high","low","adjclose","symbol"]])

       fecha      dato
328  2019-03  19.24774
-------
          date   open      close       high        low   adjclose symbol
14  2019-03-29  73.18  74.120003  74.190002  72.910004  74.120003   AABA
-------
         date         open        close         high          low  \
14 2019-03-29  1408.549604  1426.642526  1427.989862  1403.352779   

       adjclose symbol  
14  1426.642526   AABA  


**e)	Borre todos aquellos registros anteriores al 01 de enero del año 2019.**

In [14]:
df_res = df_a2.drop( df_a2[(pd.DatetimeIndex(df_a2["date"]).year < 2019)].index).reset_index(drop=True) #########

In [15]:
print(df_a2.count())
print(df_a2[(pd.DatetimeIndex(df_a2["date"]).year < 2019)].count())
print(df_res.count())

date        10182789
volume      10182789
open        10182789
close       10182789
high        10182789
low         10182789
adjclose    10182789
symbol      10182789
dtype: int64
date        9943488
volume      9943488
open        9943488
close       9943488
high        9943488
low         9943488
adjclose    9943488
symbol      9943488
dtype: int64
date        239301
volume      239301
open        239301
close       239301
high        239301
low         239301
adjclose    239301
symbol      239301
dtype: int64


**f)	Si tiene algunas observaciones anótelas a modo de conclusiones en el notebook.**

Solo tuve una observación: <br>
* En el punto d) <em>Transforme los valores monetarios de dólares a pesos (Ocupe el promedio mensual de los tipos de cambio para ello)</em><br>
Tome como valores monetarios los que tenian punto decimal.

**g)	Exporte el DataFrame resultante en un archivo csv.**

In [16]:
df_res.to_csv(path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop', "parteA.csv"), index=False)